In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
import torch
import random
from torch.utils.data import Dataset, DataLoader
from datasets_dataloader_pytorch import CustomDataset, load_data
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, confusion_matrix
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, average_precision_score

from variants import CAMELOT_GMM, class_weight, Camelot_GRU, Camelot_denoising
from model_utils import MyLRScheduler, calc_l1_l2_loss
from utils import calc_pred_loss, calc_dist_loss, calc_clus_loss, torch_log

metrics = ['AUC', 'F1 score', 'Recall', 'NMI']
seeds = [1001, 1012, 1134, 2475, 6138, 7415, 1663, 7205, 9253, 1782]

In [2]:
results = np.zeros((len(seeds), 4))
for index, SEED in enumerate(seeds):
    torch.random.manual_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    dataset = CustomDataset(time_range=(0, 10))

    # Stratified Sampling for train and val
    train_idx, test_idx = train_test_split(np.arange(len(dataset)),
                                                test_size=0.4,
                                                random_state=SEED,
                                                shuffle=True,
                                                stratify=np.argmax(dataset.y,axis=-1))

    # Subset dataset for train and val
    train_val_dataset = dataset.get_subset(train_idx)
    test_dataset = dataset.get_subset(test_idx)

    train_idx,  val_idx = train_test_split(np.arange(len(train_val_dataset)),
                                                test_size=0.4,
                                                random_state=SEED,
                                                shuffle=True,
                                                stratify=np.argmax(train_val_dataset.y,axis=-1))

    train_dataset = train_val_dataset.get_subset(train_idx)
    val_dataset = train_val_dataset.get_subset(val_idx)

    train_loader, val_loader, test_loader = load_data(train_dataset, val_dataset, test_dataset)

    model = CAMELOT_GMM(input_shape=(train_dataset.x.shape[1], train_dataset.x.shape[2]), seed=SEED, num_clusters=10, latent_dim=64)
    model = model.to(device)

    train_x = torch.tensor(train_dataset.x).to(device)
    train_y = torch.tensor(train_dataset.y).to(device)
    val_x = torch.tensor(val_dataset.x).to(device)
    val_y = torch.tensor(val_dataset.y).to(device)

    model.initialize((train_x, train_y), (val_x, val_y))

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    cluster_optim = torch.optim.Adam([model.cluster_rep_set], lr=0.001)

    lr_scheduler = MyLRScheduler(optimizer, patience=15, min_lr=0.00001, factor=0.25)
    cluster_lr_scheduler = MyLRScheduler(cluster_optim, patience=15, min_lr=0.00001, factor=0.25)

    loss_mat = np.zeros((100, 4, 2))

    best_loss = 1e5
    count = 0
    for i in trange(100):
        for step, (x_train, y_train) in enumerate(train_loader):
            optimizer.zero_grad()
            cluster_optim.zero_grad()
            
            y_pred, probs = model.forward_pass(x_train)
            
            loss_weights = class_weight(y_train)
            
            common_loss = calc_pred_loss(y_train, y_pred, loss_weights)
            
            enc_loss = common_loss + model.alpha * calc_dist_loss(probs) + \
            + calc_l1_l2_loss(part=model.Encoder) 
            enc_loss.backward(retain_graph=True, inputs=list(model.Encoder.parameters()))
            
            idnetifier_loss = common_loss + model.alpha * calc_dist_loss(probs) + \
            + calc_l1_l2_loss(layers=[model.Identifier.fc2])
            idnetifier_loss.backward(retain_graph=True, inputs=list(model.Identifier.parameters()))
            
            pred_loss = common_loss + calc_l1_l2_loss(layers=[model.Predictor.fc2, model.Predictor.fc3])
            pred_loss.backward(retain_graph=True, inputs=list(model.Predictor.parameters()))
            
            clus_loss = common_loss + model.beta * calc_clus_loss(model.cluster_rep_set)
            clus_loss.backward(inputs=model.cluster_rep_set)

            optimizer.step()
            cluster_optim.step()

            loss_mat[i, 0, 0] += enc_loss.item()
            loss_mat[i, 1, 0] += idnetifier_loss.item()
            loss_mat[i, 2, 0] += pred_loss.item()
            loss_mat[i, 3, 0] += clus_loss.item()
                            
        with torch.no_grad():
            for step, (x_val, y_val) in enumerate(val_loader):
                y_pred, probs = model.forward_pass(x_val)
                
                loss_weights = class_weight(y_val)
                
                common_loss = calc_pred_loss(y_val, y_pred, loss_weights)
            
                enc_loss = common_loss + model.alpha * calc_dist_loss(probs) + \
                + calc_l1_l2_loss(part=model.Encoder) 

                idnetifier_loss = common_loss + model.alpha * calc_dist_loss(probs) + \
                + calc_l1_l2_loss(layers=[model.Identifier.fc2])

                pred_loss = common_loss + calc_l1_l2_loss(layers=[model.Predictor.fc2, model.Predictor.fc3])

                clus_loss = common_loss + model.beta * calc_clus_loss(model.cluster_rep_set)
                
                loss_mat[i, 0, 1] += enc_loss.item()
                loss_mat[i, 1, 1] += idnetifier_loss.item()
                loss_mat[i, 2, 1] += pred_loss.item()
                loss_mat[i, 3, 1] += clus_loss.item()
            
            if i >= 30:
                if loss_mat[i, 0, 1] < best_loss:
                    count = 0
                    best_loss = loss_mat[i, 0, 1]
                    torch.save(model.state_dict(), './best_model')
                else:
                    count += 1
                    if count >= 50:
                        model.load_state_dict(torch.load('./best_model'))
        lr_scheduler.step(loss_mat[i, 0, 1])
        cluster_lr_scheduler.step(loss_mat[i, 0, 1])

    model.load_state_dict(torch.load('./best_model'))

    real, preds = [], []
    with torch.no_grad():
        for _, (x, y) in enumerate(test_loader):
            y_pred, _ = model.forward_pass(x)
            preds.extend(list(y_pred.cpu().detach().numpy()))
            real.extend(list(y.cpu().detach().numpy()))

    auc = roc_auc_score(real, preds, average=None)

    labels_true, labels_pred = np.argmax(real, axis=1), np.argmax(preds, axis=1)

    # Compute F1
    f1 = f1_score(labels_true, labels_pred, average=None)

    # Compute Recall
    rec = recall_score(labels_true, labels_pred, average=None)

    # Compute NMI
    nmi = normalized_mutual_info_score(labels_true, labels_pred)

    print(f'AUCROC: \t{auc.mean():.5f}, \t{auc}')
    print(f'F1-score: \t{f1.mean():.5f}, \t{f1}')
    print(f'Recall: \t{rec.mean():.5f}, \t{rec}')
    print(f'NMI: \t\t{nmi:.5f}')
    
    results[index, 0] = auc.mean()
    results[index, 1] = f1.mean()
    results[index, 2] = rec.mean()
    results[index, 3] = nmi


MIMIC data has been subsettted to the following features: 
 ['DBP', 'ESI', 'HR', 'RR', 'SBP', 'SPO2', 'TEMP', 'age', 'gender'].


 50%|█████     | 50/100 [00:13<00:13,  3.62it/s]


Encoder initialization done!
GMM initialization done!
Cluster initialization done!


 50%|█████     | 50/100 [00:11<00:11,  4.37it/s]


Identifier initialization done!


...

Encoder initialization done!
GMM initialization done!
Cluster initialization done!


 50%|█████     | 50/100 [00:11<00:11,  4.50it/s]


Identifier initialization done!


100%|██████████| 100/100 [00:51<00:00,  1.94it/s]

AUCROC: 	0.77340, 	[0.8854296  0.7782953  0.75083331 0.67904015]
F1-score: 	0.31077, 	[0.         0.49898854 0.6875308  0.05657238]
Recall: 	0.36231, 	[0.         0.56748466 0.59361702 0.28813559]
NMI: 		0.09783


In [3]:
for m, u, std in zip(metrics, results.mean(axis=0), results.std(axis=0)):
    print(f'{m}: {u:.3f} ({std:.3f})')

AUC: 0.768 (0.019)
F1 score: 0.324 (0.012)
Recall: 0.353 (0.006)
NMI: 0.107 (0.006)


In [3]:
results = np.zeros((len(seeds), 4))
for index, SEED in enumerate(seeds):
    torch.random.manual_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    dataset = CustomDataset(time_range=(0, 10))

    # Stratified Sampling for train and val
    train_idx, test_idx = train_test_split(np.arange(len(dataset)),
                                                test_size=0.4,
                                                random_state=SEED,
                                                shuffle=True,
                                                stratify=np.argmax(dataset.y,axis=-1))

    # Subset dataset for train and val
    train_val_dataset = dataset.get_subset(train_idx)
    test_dataset = dataset.get_subset(test_idx)

    train_idx,  val_idx = train_test_split(np.arange(len(train_val_dataset)),
                                                test_size=0.4,
                                                random_state=SEED,
                                                shuffle=True,
                                                stratify=np.argmax(train_val_dataset.y,axis=-1))

    train_dataset = train_val_dataset.get_subset(train_idx)
    val_dataset = train_val_dataset.get_subset(val_idx)

    train_loader, val_loader, test_loader = load_data(train_dataset, val_dataset, test_dataset)

    model = Camelot_GRU(input_shape=(train_dataset.x.shape[1], train_dataset.x.shape[2]), seed=SEED, num_clusters=10, latent_dim=64)
    model = model.to(device)

    train_x = torch.tensor(train_dataset.x).to(device)
    train_y = torch.tensor(train_dataset.y).to(device)
    val_x = torch.tensor(val_dataset.x).to(device)
    val_y = torch.tensor(val_dataset.y).to(device)

    model.initialize((train_x, train_y), (val_x, val_y))

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    cluster_optim = torch.optim.Adam([model.cluster_rep_set], lr=0.001)

    lr_scheduler = MyLRScheduler(optimizer, patience=15, min_lr=0.00001, factor=0.25)
    cluster_lr_scheduler = MyLRScheduler(cluster_optim, patience=15, min_lr=0.00001, factor=0.25)

    loss_mat = np.zeros((100, 4, 2))

    best_loss = 1e5
    count = 0
    for i in trange(100):
        for step, (x_train, y_train) in enumerate(train_loader):
            optimizer.zero_grad()
            cluster_optim.zero_grad()
            
            y_pred, probs = model.forward_pass(x_train)
            
            loss_weights = class_weight(y_train)
            
            common_loss = calc_pred_loss(y_train, y_pred, loss_weights)
            
            enc_loss = common_loss + model.alpha * calc_dist_loss(probs) + \
            + calc_l1_l2_loss(part=model.Encoder) 
            enc_loss.backward(retain_graph=True, inputs=list(model.Encoder.parameters()))
            
            idnetifier_loss = common_loss + model.alpha * calc_dist_loss(probs) + \
            + calc_l1_l2_loss(layers=[model.Identifier.fc2])
            idnetifier_loss.backward(retain_graph=True, inputs=list(model.Identifier.parameters()))
            
            pred_loss = common_loss + calc_l1_l2_loss(layers=[model.Predictor.fc2, model.Predictor.fc3])
            pred_loss.backward(retain_graph=True, inputs=list(model.Predictor.parameters()))
            
            clus_loss = common_loss + model.beta * calc_clus_loss(model.cluster_rep_set)
            clus_loss.backward(inputs=model.cluster_rep_set)

            optimizer.step()
            cluster_optim.step()

            loss_mat[i, 0, 0] += enc_loss.item()
            loss_mat[i, 1, 0] += idnetifier_loss.item()
            loss_mat[i, 2, 0] += pred_loss.item()
            loss_mat[i, 3, 0] += clus_loss.item()
                            
        with torch.no_grad():
            for step, (x_val, y_val) in enumerate(val_loader):
                y_pred, probs = model.forward_pass(x_val)
                
                loss_weights = class_weight(y_val)
                
                common_loss = calc_pred_loss(y_val, y_pred, loss_weights)
            
                enc_loss = common_loss + model.alpha * calc_dist_loss(probs) + \
                + calc_l1_l2_loss(part=model.Encoder) 

                idnetifier_loss = common_loss + model.alpha * calc_dist_loss(probs) + \
                + calc_l1_l2_loss(layers=[model.Identifier.fc2])

                pred_loss = common_loss + calc_l1_l2_loss(layers=[model.Predictor.fc2, model.Predictor.fc3])

                clus_loss = common_loss + model.beta * calc_clus_loss(model.cluster_rep_set)
                
                loss_mat[i, 0, 1] += enc_loss.item()
                loss_mat[i, 1, 1] += idnetifier_loss.item()
                loss_mat[i, 2, 1] += pred_loss.item()
                loss_mat[i, 3, 1] += clus_loss.item()
            
            if i >= 30:
                if loss_mat[i, 0, 1] < best_loss:
                    count = 0
                    best_loss = loss_mat[i, 0, 1]
                    torch.save(model.state_dict(), './best_model')
                else:
                    count += 1
                    if count >= 50:
                        model.load_state_dict(torch.load('./best_model'))
        lr_scheduler.step(loss_mat[i, 0, 1])
        cluster_lr_scheduler.step(loss_mat[i, 0, 1])

    model.load_state_dict(torch.load('./best_model'))

    real, preds = [], []
    with torch.no_grad():
        for _, (x, y) in enumerate(test_loader):
            y_pred, _ = model.forward_pass(x)
            preds.extend(list(y_pred.cpu().detach().numpy()))
            real.extend(list(y.cpu().detach().numpy()))

    auc = roc_auc_score(real, preds, average=None)

    labels_true, labels_pred = np.argmax(real, axis=1), np.argmax(preds, axis=1)

    # Compute F1
    f1 = f1_score(labels_true, labels_pred, average=None)

    # Compute Recall
    rec = recall_score(labels_true, labels_pred, average=None)

    # Compute NMI
    nmi = normalized_mutual_info_score(labels_true, labels_pred)

    print(f'AUCROC: \t{auc.mean():.5f}, \t{auc}')
    print(f'F1-score: \t{f1.mean():.5f}, \t{f1}')
    print(f'Recall: \t{rec.mean():.5f}, \t{rec}')
    print(f'NMI: \t\t{nmi:.5f}')
    
    results[index, 0] = auc.mean()
    results[index, 1] = f1.mean()
    results[index, 2] = rec.mean()
    results[index, 3] = nmi


MIMIC data has been subsettted to the following features: 
 ['DBP', 'ESI', 'HR', 'RR', 'SBP', 'SPO2', 'TEMP', 'age', 'gender'].


 50%|█████     | 50/100 [00:13<00:13,  3.69it/s]


Encoder initialization done!
Kmeans initialization done!
Cluster initialization done!


 50%|█████     | 50/100 [00:11<00:11,  4.42it/s]


Identifier initialization done!


...

Encoder initialization done!
Kmeans initialization done!
Cluster initialization done!


 50%|█████     | 50/100 [00:11<00:11,  4.49it/s]


Identifier initialization done!


100%|██████████| 100/100 [00:50<00:00,  1.98it/s]

AUCROC: 	0.74896, 	[0.84658608 0.77079771 0.74461885 0.63384054]
F1-score: 	0.32546, 	[0.         0.49759615 0.80425436 0.        ]
Recall: 	0.34778, 	[0.         0.63496933 0.75617021 0.        ]
NMI: 		0.09260


In [6]:
for m, u, std in zip(metrics, results.mean(axis=0), results.std(axis=0)):
    print(f'{m}: {u:.3f} ({std:.3f})')

AUC: 0.764 (0.019)
F1 score: 0.323 (0.010)
Recall: 0.364 (0.022)
NMI: 0.101 (0.010)


In [2]:
results = np.zeros((len(seeds), 4))
for index, SEED in enumerate(seeds):
    torch.random.manual_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    dataset = CustomDataset(time_range=(0, 10))

    # Stratified Sampling for train and val
    train_idx, test_idx = train_test_split(np.arange(len(dataset)),
                                                test_size=0.4,
                                                random_state=SEED,
                                                shuffle=True,
                                                stratify=np.argmax(dataset.y,axis=-1))

    # Subset dataset for train and val
    train_val_dataset = dataset.get_subset(train_idx)
    test_dataset = dataset.get_subset(test_idx)

    train_idx,  val_idx = train_test_split(np.arange(len(train_val_dataset)),
                                                test_size=0.4,
                                                random_state=SEED,
                                                shuffle=True,
                                                stratify=np.argmax(train_val_dataset.y,axis=-1))

    train_dataset = train_val_dataset.get_subset(train_idx)
    val_dataset = train_val_dataset.get_subset(val_idx)

    train_loader, val_loader, test_loader = load_data(train_dataset, val_dataset, test_dataset)

    model = Camelot_denoising(input_shape=(train_dataset.x.shape[1], train_dataset.x.shape[2]), seed=SEED, num_clusters=10, latent_dim=64)
    model = model.to(device)

    train_x = torch.tensor(train_dataset.x).to(device)
    train_y = torch.tensor(train_dataset.y).to(device)
    val_x = torch.tensor(val_dataset.x).to(device)
    val_y = torch.tensor(val_dataset.y).to(device)

    model.initialize((train_x, train_y), (val_x, val_y))

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    cluster_optim = torch.optim.Adam([model.cluster_rep_set], lr=0.001)

    lr_scheduler = MyLRScheduler(optimizer, patience=15, min_lr=0.00001, factor=0.25)
    cluster_lr_scheduler = MyLRScheduler(cluster_optim, patience=15, min_lr=0.00001, factor=0.25)

    loss_mat = np.zeros((100, 4, 2))

    best_loss = 1e5
    count = 0
    for i in trange(100):
        for step, (x_train, y_train) in enumerate(train_loader):
            optimizer.zero_grad()
            cluster_optim.zero_grad()
            
            y_pred, probs = model.forward_pass(x_train)
            
            loss_weights = class_weight(y_train)
            
            common_loss = calc_pred_loss(y_train, y_pred, loss_weights)
            
            enc_loss = common_loss + model.alpha * calc_dist_loss(probs) + \
            + calc_l1_l2_loss(part=model.Encoder) 
            enc_loss.backward(retain_graph=True, inputs=list(model.Encoder.parameters()))
            
            idnetifier_loss = common_loss + model.alpha * calc_dist_loss(probs) + \
            + calc_l1_l2_loss(layers=[model.Identifier.fc2])
            idnetifier_loss.backward(retain_graph=True, inputs=list(model.Identifier.parameters()))
            
            pred_loss = common_loss + calc_l1_l2_loss(layers=[model.Predictor.fc2, model.Predictor.fc3])
            pred_loss.backward(retain_graph=True, inputs=list(model.Predictor.parameters()))
            
            clus_loss = common_loss + model.beta * calc_clus_loss(model.cluster_rep_set)
            clus_loss.backward(inputs=model.cluster_rep_set)

            optimizer.step()
            cluster_optim.step()

            loss_mat[i, 0, 0] += enc_loss.item()
            loss_mat[i, 1, 0] += idnetifier_loss.item()
            loss_mat[i, 2, 0] += pred_loss.item()
            loss_mat[i, 3, 0] += clus_loss.item()
                            
        with torch.no_grad():
            for step, (x_val, y_val) in enumerate(val_loader):
                y_pred, probs = model.forward_pass(x_val)
                
                loss_weights = class_weight(y_val)
                
                common_loss = calc_pred_loss(y_val, y_pred, loss_weights)
            
                enc_loss = common_loss + model.alpha * calc_dist_loss(probs) + \
                + calc_l1_l2_loss(part=model.Encoder) 

                idnetifier_loss = common_loss + model.alpha * calc_dist_loss(probs) + \
                + calc_l1_l2_loss(layers=[model.Identifier.fc2])

                pred_loss = common_loss + calc_l1_l2_loss(layers=[model.Predictor.fc2, model.Predictor.fc3])

                clus_loss = common_loss + model.beta * calc_clus_loss(model.cluster_rep_set)
                
                loss_mat[i, 0, 1] += enc_loss.item()
                loss_mat[i, 1, 1] += idnetifier_loss.item()
                loss_mat[i, 2, 1] += pred_loss.item()
                loss_mat[i, 3, 1] += clus_loss.item()
            
            if i >= 30:
                if loss_mat[i, 0, 1] < best_loss:
                    count = 0
                    best_loss = loss_mat[i, 0, 1]
                    torch.save(model.state_dict(), './best_model')
                else:
                    count += 1
                    if count >= 50:
                        model.load_state_dict(torch.load('./best_model'))
        lr_scheduler.step(loss_mat[i, 0, 1])
        cluster_lr_scheduler.step(loss_mat[i, 0, 1])

    model.load_state_dict(torch.load('./best_model'))

    real, preds = [], []
    with torch.no_grad():
        for _, (x, y) in enumerate(test_loader):
            y_pred, _ = model.forward_pass(x)
            preds.extend(list(y_pred.cpu().detach().numpy()))
            real.extend(list(y.cpu().detach().numpy()))

    auc = roc_auc_score(real, preds, average=None)

    labels_true, labels_pred = np.argmax(real, axis=1), np.argmax(preds, axis=1)

    # Compute F1
    f1 = f1_score(labels_true, labels_pred, average=None)

    # Compute Recall
    rec = recall_score(labels_true, labels_pred, average=None)

    # Compute NMI
    nmi = normalized_mutual_info_score(labels_true, labels_pred)

    print(f'AUCROC: \t{auc.mean():.5f}, \t{auc}')
    print(f'F1-score: \t{f1.mean():.5f}, \t{f1}')
    print(f'Recall: \t{rec.mean():.5f}, \t{rec}')
    print(f'NMI: \t\t{nmi:.5f}')
    
    results[index, 0] = auc.mean()
    results[index, 1] = f1.mean()
    results[index, 2] = rec.mean()
    results[index, 3] = nmi


MIMIC data has been subsettted to the following features: 
 ['DBP', 'ESI', 'HR', 'RR', 'SBP', 'SPO2', 'TEMP', 'age', 'gender'].


 50%|█████     | 50/100 [00:14<00:14,  3.37it/s]


Encoder initialization done!
Kmeans initialization done!
Cluster initialization done!


 50%|█████     | 50/100 [00:12<00:12,  3.95it/s]


Identifier initialization done!


...

Encoder initialization done!
Kmeans initialization done!
Cluster initialization done!


 50%|█████     | 50/100 [00:11<00:11,  4.43it/s]


Identifier initialization done!


100%|██████████| 100/100 [00:52<00:00,  1.89it/s]

AUCROC: 	0.76780, 	[0.90457367 0.77043117 0.74609745 0.65009142]
F1-score: 	0.32388, 	[0.         0.49182314 0.80370036 0.        ]
Recall: 	0.34514, 	[0.         0.62269939 0.75787234 0.        ]
NMI: 		0.09293


In [3]:
for m, u, std in zip(metrics, results.mean(axis=0), results.std(axis=0)):
    print(f'{m}: {u:.3f} ({std:.3f})')

AUC: 0.764 (0.009)
F1 score: 0.320 (0.013)
Recall: 0.362 (0.036)
NMI: 0.103 (0.011)
